In [20]:
import os
import fitz  # PyMuPDF
from dotenv import load_dotenv

from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# 🔐 환경변수 로드
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")


# ---------------------------------------------------
# ✅ 1. PDF → 좌/우 텍스트 추출 → TXT 저장
# ---------------------------------------------------

def extract_left_right_text(pdf_path):
    doc = fitz.open(pdf_path)
    left_text_total = ""
    right_text_total = ""

    for page_num, page in enumerate(doc):
        width = page.rect.width
        blocks = page.get_text("blocks")

        left_text = ""
        right_text = ""

        for b in blocks:
            x0, y0, x1, y1, text, *_ = b
            center_x = (x0 + x1) / 2
            if center_x < width / 2:
                left_text += text.strip() + " "
            else:
                right_text += text.strip() + " "

        left_text_total += f"[페이지 {page_num + 1} - 좌]\n{left_text.strip()}\n\n"
        right_text_total += f"[페이지 {page_num + 1} - 우]\n{right_text.strip()}\n\n"

    return left_text_total + right_text_total  # ⬅️ 합쳐서 반환


# 🔹 PDF 경로 및 TXT 저장
pdf_path = "KB_실버암_간편건강보험Plus.pdf"
txt_path = "KB_실버암_간편건강보험Plus.txt"

if not os.path.exists(txt_path):
    full_text = extract_left_right_text(pdf_path)
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(full_text)
    print(f"✅ 텍스트 저장 완료: {txt_path}")
else:
    print(f"📄 기존 텍스트 파일 사용: {txt_path}")


# ---------------------------------------------------
# ✅ 2. LangChain RAG 파이프라인
# ---------------------------------------------------

# 📄 TXT 로드 및 분할
loader = TextLoader(txt_path, encoding="utf-8")
docs_raw = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(docs_raw)

# 🧠 임베딩 및 벡터스토어
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever()

# 🤖 Claude LLM
llm = ChatAnthropic(
    model="claude-opus-4-20250514",
    temperature=0,
    max_tokens=1024,
    api_key=ANTHROPIC_API_KEY,
)

# 🧩 RAG 프롬프트 + 체인 구성
prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

llm_only_chain = llm | StrOutputParser()

📄 기존 텍스트 파일 사용: KB_실버암_간편건강보험Plus.txt


C:\Users\KDT15\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [21]:
# ---------------------------------------------------
# ✅ 3. 질문 실행
# ---------------------------------------------------
# 질문
# question = "고지의무를 위반한 경우 보험사는 어떤 조치를 취할 수 있나요?"                                  # 민석 질문
# question = "특정암 ii란 뭐야?"                                                                          # 1번 질문
# question = "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?"                # 2번 질문                                                   # 3번 질문
question = "통합전이암진단비의 보장의범위는 어떻게 돼?"                                                     # 4번 질문
# question = "보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?"      # 5번 질문
# question = "신재진단암진단비ii 에서 수술이란 뭐야?"                                                       # 6번 질문    
# question = "보험금청구 했는데 언제 지급되니?"      


# 1. RAG 기반 응답
rag_answer = rag_chain.invoke(question)

# 2. LLM 단독 응답
llm_only_answer = llm_only_chain.invoke(question)

# 출력
print("🧾 질문:", question)
print("\n💬 [LLM 단독 응답]:")
print(llm_only_answer)
print("\n📚 [RAG 기반 응답]:")
print(rag_answer)

🧾 질문: 통합전이암진단비의 보장의범위는 어떻게 돼?

💬 [LLM 단독 응답]:
통합전이암진단비의 보장범위에 대해 설명드리겠습니다.

## 통합전이암진단비 보장범위

### 1. **기본 보장내용**
- 원발암과 다른 장기로 암이 전이된 경우
- 최초 1회에 한해 진단비 지급
- 원발암 진단 이후 전이암 진단 시 보장

### 2. **주요 보장 조건**
- **원격전이**: 원발 부위에서 멀리 떨어진 다른 장기로의 전이
- **진단확정**: 병리학적 검사로 확진된 경우
- **면책기간**: 보통 계약일로부터 90일 이후 보장

### 3. **보장 제외사항**
- 원발암의 재발
- 인접장기로의 직접 침윤
- 림프절 전이만 있는 경우 (일부 상품)
- 면책기간 내 진단

### 4. **보험사별 차이점**
- 보장금액: 1,000만원~5,000만원 수준
- 세부 약관 조건 상이
- 갱신/비갱신 여부

**정확한 보장범위는 가입하신 보험상품의 약관을 확인하시거나, 보험사에 직접 문의하시는 것이 가장 정확합니다.**

📚 [RAG 기반 응답]:
통합전이암진단비는 보험책임개시일 이후 통합전이암으로 진단확정된 경우 암구분에 따라 각각 최초 1회의 진단에 한하여 보험금을 지급합니다. 보장범위에는 특정소액전이암, 생식기전이암 및 비뇨기관전이암(자궁,전립선,방광제외), 소화기관전이암, 11대특정전이암, 폐전이암, 췌장전이암, 3대특정고액전이암 등이 포함됩니다. 각 전이암 종류별로 보험가입금액의 50%~100%가 지급되며, 가입 후 경과기간에 따라 지급률이 달라집니다.
